In [2]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout
import glob
import matplotlib.pyplot as plt
import cv2
import numpy as np
import keras


test_dir = '/content/drive/MyDrive/KASHIF DEEP LEARNING/transfer learning/ Jason Brownlee/dataBase/test'

validation_dir = '/content/drive/MyDrive/KASHIF DEEP LEARNING/transfer learning/ Jason Brownlee/dataBase/val'

train_dir = '/content/drive/MyDrive/KASHIF DEEP LEARNING/transfer learning/ Jason Brownlee/dataBase/train'

In [3]:
image_size=[263,173]


In [4]:
def dataset(train_path,test_path):
    datagen = ImageDataGenerator()
    train_set = datagen.flow_from_directory(train_path, class_mode='categorical', batch_size=20,target_size=image_size)
    test_set = datagen.flow_from_directory(test_path, class_mode='categorical', batch_size=20,target_size=image_size)
    return train_set,test_set

In [5]:
def model_creation(model_name):
    
    vgg_model = model_name(weights='imagenet', include_top=False, input_shape=image_size + [3])
    
    for layer in vgg_model.layers:
        layer.trainable=False    



    xx=Dropout(rate=0.20)(vgg_model.output)
    x=Flatten()(xx)
    prediction=Dense(12,activation="softmax")(x)
    model=Model(inputs=vgg_model.input , outputs=prediction)
    model.summary()
    
    return model

In [6]:

# for layer in model.layers:
#     print(layer, layer.trainable)


In [7]:

model=model_creation(VGG16)


58900480/58889256 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 263, 173, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 263, 173, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 263, 173, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 131, 86, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 131, 86, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 131, 86, 128)      147584    
______________________________________________________________

In [8]:

for layer in model.layers:
    print(layer, layer.trainable)


<keras.engine.input_layer.InputLayer object at 0x7f018c65a050> False
<keras.layers.convolutional.Conv2D object at 0x7f012a652290> False
<keras.layers.convolutional.Conv2D object at 0x7f012a679f50> False
<keras.layers.pooling.MaxPooling2D object at 0x7f0129dbfb50> False
<keras.layers.convolutional.Conv2D object at 0x7f0120529b50> False
<keras.layers.convolutional.Conv2D object at 0x7f0120531f50> False
<keras.layers.pooling.MaxPooling2D object at 0x7f0120533a10> False
<keras.layers.convolutional.Conv2D object at 0x7f0120538950> False
<keras.layers.convolutional.Conv2D object at 0x7f01205418d0> False
<keras.layers.convolutional.Conv2D object at 0x7f0120533cd0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f0120549310> False
<keras.layers.convolutional.Conv2D object at 0x7f01204ce4d0> False
<keras.layers.convolutional.Conv2D object at 0x7f018913b650> False
<keras.layers.convolutional.Conv2D object at 0x7f01204d62d0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f01204e0350>

In [9]:
train=train_dir
test=validation_dir
train_set,test_set=dataset(train,test)


Found 4200 images belonging to 12 classes.
Found 1200 images belonging to 12 classes.


In [10]:
import tensorflow as tf
# filepath = '/content/drive/MyDrive/KASHIF DEEP LEARNING/transfer learning/ Jason Brownlee/dataBase/George zoto   97/temp_saved'
filepath  = '/content/drive/MyDrive/KASHIF DEEP LEARNING/transfer learning/ Jason Brownlee/dataBase/here'



checkpoint_filepath = '/content/drive/MyDrive/KASHIF DEEP LEARNING/transfer learning/ Jason Brownlee/dataBase/here/check.h5'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose = 1,
    save_best_only=True)









# call = tf.keras.callbacks.ModelCheckpoint(
#     filepath,
#     monitor="val_loss",
#     verbose=0,
#     save_best_only=True,
#     save_weights_only=False,
#     mode="auto",
#     save_freq="epoch",
#     options=None,
# )




In [11]:
model.compile(loss="categorical_crossentropy",
              optimizer='sgd',
              metrics=['accuracy'] ,)




In [12]:
import tensorflow as tf

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)



In [18]:
# from keras.optimizers import SGD
# tf.keras.optimizers.SGD(
#     learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD", **kwargs
# )



# from keras.optimizers import sgd
model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True) ,
              metrics=['accuracy'] ,)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [19]:




history=model.fit(train_set,
                  validation_data=test_set,
                  epochs=5,
                 callbacks = [model_checkpoint_callback , callback],
                  shuffle=True ,
                )

Epoch 1/5
210/210 [==============================] - 1254s 6s/step - loss: 97.5781 - accuracy: 0.8281 - val_loss: 27.8889 - val_accuracy: 0.9317

Epoch 00001: val_accuracy improved from -inf to 0.93167, saving model to /content/drive/MyDrive/KASHIF DEEP LEARNING/transfer learning/ Jason Brownlee/dataBase/here/check.h5
Epoch 2/5
210/210 [==============================] - 56s 265ms/step - loss: 27.3282 - accuracy: 0.9419 - val_loss: 22.7960 - val_accuracy: 0.9475

Epoch 00002: val_accuracy improved from 0.93167 to 0.94750, saving model to /content/drive/MyDrive/KASHIF DEEP LEARNING/transfer learning/ Jason Brownlee/dataBase/here/check.h5
Epoch 3/5
210/210 [==============================] - 56s 267ms/step - loss: 17.4216 - accuracy: 0.9614 - val_loss: 32.3361 - val_accuracy: 0.9417

Epoch 00003: val_accuracy did not improve from 0.94750
Epoch 4/5
210/210 [==============================] - 56s 266ms/step - loss: 10.9039 - accuracy: 0.9760 - val_loss: 10.5860 - val_accuracy: 0.9758

Epoch 0

In [21]:
model.save('/content/drive/MyDrive/KASHIF DEEP LEARNING/transfer learning/ Jason Brownlee/dataBase/George 97.33')   

INFO:tensorflow:Assets written to: /content/drive/MyDrive/KASHIF DEEP LEARNING/transfer learning/ Jason Brownlee/dataBase/George 97.33/assets


In [22]:
from keras.models import load_model

vgg16_saved = load_model('/content/drive/MyDrive/KASHIF DEEP LEARNING/transfer learning/ Jason Brownlee/dataBase/George 97.33')

In [23]:

test_datagen = ImageDataGenerator( rescale = 1.0/255. )

test_generator =  test_datagen.flow_from_directory( test_dir,
                                                          batch_size  = 32,
                                                          class_mode  = 'categorical', 
                                                          target_size = image_size)

Found 600 images belonging to 12 classes.


In [24]:
predict_vgg16  =vgg16_saved.evaluate(test_generator, 
                                    batch_size=None,
                                    verbose=1, 
                                    steps=600/32,
                                    callbacks=None,
                                    max_queue_size=10, 
                                    workers=1, use_multiprocessing=False)

18/18 [==============================] - 131s 7s/step - loss: 14.2695 - accuracy: 0.6350
